### Casting and Reference Conversions

An object reference can be:  
- ***Implicitly*** `upcast` to a base class reference
- ***Explicitly*** `downcast` to a subclass reference

`Upcasting` and `downcasting` between compatible reference types performs ***reference conversions***

#### Upcasting

In [3]:
public class Asset
{
    public string Name;
}

public class Stock : Asset // inherits from Asset
{
    public long SharesOwned;
}

public class House : Asset // inherits from Asset
{
    public decimal Mortgage;
}

Stock msft = new Stock();
Asset a = msft; // Upcast

Console.WriteLine (a == msft); // true

Console.WriteLine (a.Name); // OK
//Console.WriteLine (a.SharesOwned); // Compile-time error

True



#### Downcasting

In [ ]:
//a downcast operation creates a subclass reference from a base class reference:
Stock msft = new Stock();
Asset a = msft;
Stock s = (Stock)a; // Downcast
Console.WriteLine (s.SharesOwned); // <No error>
Console.WriteLine (s == a); // True
Console.WriteLine (s == msft); // True

House h = new House();
Asset aa = h; // Upcast always succeeds
Stock ss = (Stock)aa; // Downcast fails: a is not a Stock

/******This is an example of runtime type checking *****/

#### The as operator

In [ ]:
//as operator is downcast, evaluate to null if downcast fails

Asset a = new Asset();
Stock s = a as Stock; // s is null; no exception thrown
if (s != null) Console.WriteLine (s.SharesOwned);

#### The is operator

In [4]:
//is operator tests whether a variable matches a pattern.
if (a is Stock)
    Console.WriteLine (((Stock)a).SharesOwned);

#### Introducing a pattern variable

In [ ]:
if (a is Stock s)
    Console.WriteLine (s.SharesOwned);

if (a is Stock ss && ss.SharesOwned > 100000)
    Console.WriteLine ("Wealthy");

In [ ]:
//it remains in scope outside the is expression

if (a is Stock s && s.SharesOwned > 100000)
Console.WriteLine ("Wealthy");
else
s = new Stock(); // s is in scope
Console.WriteLine (s.SharesOwned); // Still in scope

### Virtual Function Members

A function marked as ***virtual*** can be `overridden` by `subclasses` wanting to provide a specialized implementation.

`Methods`, `properties`, `indexers`, and `events` can all be declared virtual

In [5]:
public class Asset
{
    public string Name;
    public virtual decimal Liability => 0; // Expression-bodied property
    //Liability => 0 is a shortcut for { get { return 0; } }.
}

public class Stock : Asset
{
    public long SharesOwned;
}

public class House : Asset
{
    public decimal Mortgage;
    public override decimal Liability => Mortgage;
}

In [ ]:
House mansion = new House { Name="McMansion", Mortgage=250000 };
Asset a = mansion;
Console.WriteLine (mansion.Liability); // 250000
Console.WriteLine (a.Liability); // 250000

#### Abstract Classes and Abstract Members

A class declared as `abstract` can `never be instantiated`.  
Abstract classes are able to define `abstract members`.  
***Abstract members*** are like ***virtual members*** except that `they don’t provide a default implementation`.

In [ ]:
public abstract class Asset
{
    // Note empty implementation
    public abstract decimal NetValue { get; }
}
public class Stock : Asset
{
    public long SharesOwned;
    public decimal CurrentPrice;
    // Override like a virtual method.
    public override decimal NetValue => CurrentPrice * SharesOwned;
}

### Hiding Inherited Members

In [ ]:
public class A { public int Counter = 1; }
public class B : A { public int Counter = 2; /*warnign is accident*/ }

public class C : A { public new int Counter = 2; /* call to other, is no accident*/ }

public class D : A { public override int Counter = 2; /* not virtual*/ }

#### new versus override

In [ ]:
public class BaseClass
{
    public virtual void Foo() { Console.WriteLine ("BaseClass.Foo"); }
}
public class Overrider : BaseClass
{
    public override void Foo() { Console.WriteLine ("Overrider.Foo"); }
}
public class Hider : BaseClass
{
    public new void Foo() { Console.WriteLine ("Hider.Foo"); }
}

Overrider over = new Overrider();
BaseClass b1 = over;
over.Foo(); // Overrider.Foo
b1.Foo(); // Overrider.Foo


Hider h = new Hider();
BaseClass b2 = h;
h.Foo(); // Hider.Foo
b2.Foo(); // BaseClass.Foo

### Sealing Functions and Classes

In [ ]:
public class BaseClass
{
    public virtual void Foo() { Console.WriteLine ("BaseClass.Foo"); }
}
public class Overrider : BaseClass
{
    public sealed override void Foo() { Console.WriteLine ("Overrider.Foo"); }
    //prevent it from being overridden by further subclasses
}
public sealed class SealedClass : BaseClass //prevent subclassing
{
    public new void Foo() { Console.WriteLine ("Hider.Foo"); }
}

### The base Keyword

The `base` keyword is similar to the `this` keyword. It serves two essential purposes:  
  
- Accessing an `overridden function member` from the `subclass`
- Calling a `base-class constructor` (see the next section)

In [ ]:
public class Asset
{
    public Asset(){}
    
    public string Name;
    public virtual decimal Liability => 0; 
}

public class House : Asset
{
    public House():base(){}

    public decimal Mortgage;
    public override decimal Liability => base.Liability + Mortgage;
}

### Constructors and Inheritance

A subclass must declare its `own constructors`. The `base class’s constructors` are
`accessible` to the `derived` class but are never `automatically` inherited.

In [2]:
public class Baseclass
{
    public int X;
    public Baseclass () { }
    public Baseclass (int x) { this.X = x; }
}

In [3]:
public class Subclass : Baseclass { }

Subclass s = new Subclass (123); //illegal

Error: (3,18): error CS1729: 'Subclass' does not contain a constructor that takes 1 arguments

In [4]:
public class Subclass : Baseclass
{
    public Subclass (int x) : base (x) { }
}

Subclass s = new Subclass (123);

Console.WriteLine(s.X);

123


***!!!Important!!!***   
Base-class constructors always `execute first`; this ensures that `base initialization` occurs `before` specialized initialization.

#### Implicit calling of the parameterless base-class constructor

If a constructor in a subclass `omits the base keyword`, the base type’s parameterless constructor is `implicitly` called

In [ ]:
public class BaseClass
{
    public int X;
    public BaseClass() { X = 1; }
}
public class Subclass : BaseClass
{
    public Subclass() { Console.WriteLine (X); } // 1
}

In [ ]:
public class BaseClass
{
    public int X;
    private BaseClass() { X = 1; }
}
public class Subclass : BaseClass
{
    public Subclass() { Console.WriteLine (X); } // 1
}

#### Constructor and field initialization order

When an object is `instantiated`, initialization takes place in the following `order`:  
  
1. From subclass to base class:  
    a. Fields are initialized.  
    b. Arguments to base-class constructor calls are evaluated.
2. From base class to subclass:  
    a. Constructor bodies execute.

In [5]:
public class B
{
    int x = 1; // Executes 3rd
    public B (int x)
    {
        // Executes 4th
    }
}
public class D : B
{
    int y = 1; // Executes 1st
    public D (int x)
        : base (x + 1) // Executes 2nd
    {
        // Executes 5th
    }
}

var d = new D(2);

### Overloading and Resolution

In [4]:
public class Asset{}
public class House : Asset{}

static void Foo (Asset a) { Console.WriteLine("Foo(Asset)"); }
static void Foo (House h) { Console.WriteLine("Foo(House)"); }

House h = new House();
Foo(h); //Foo(House)

Asset a = new House();
Foo(a); //Foo(Asset)


Foo(House)
Foo(Asset)
